![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Advanced Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics


# Von Spannungen, Verzerrungen und Verschiebungen zu Setzungen -- direkte Setzungsberechnungen

Die folgenden Überlegungen dienen der Darstellung grundlegender mechanischer Zusammenhänge, bevor die praktischen Setzungsberechnungsverfahren eingeführt werden.

In [1]:
#HIDDEN
import numpy as np #numerical methods
import matplotlib.pyplot as plt #plotting
import seaborn as sns
import sympy as sp
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import interactive
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/settlements_plots.ipynb
plt.rcParams['axes.spines.top'] = False

## Wiederholung: Lineare Elastizität in 3D

Die Effektivspannungsänderungen, die durch den Lasteintrag hervorgerufen werden, gehen mit Verformungen einher. Die Verbindung von Spannungs- und Verzerrungsänderungen erfolgt über konstitutive Modelle, also Material- oder Stoffmodelle.

Der linear elastische Festkörper wird im dreidimensionalen Raum beschrieben durch das generalisierte Hooke'sche Gesetz:

\begin{align}
\left(
    \begin{array}{c}
    \epsilon_{xx}
    \\
    \epsilon_{yy}
    \\
    \epsilon_{zz}
    \\
    \epsilon_{xy}
    \\
    \epsilon_{yz}
    \\
    \epsilon_{xz}
    \end{array}
\right)
=
\frac{1}{E}
\left(
    \begin{array}{cccccc}
    1 & -\nu & -\nu & 0 & 0 & 0
    \\
    -\nu & 1 & -\nu & 0 & 0 & 0
    \\
    -\nu & -\nu & 1 & 0 & 0 & 0
    \\
    0 & 0 & 0 & 1 + \nu & 0 & 0
    \\
    0 & 0 & 0 & 0 & 1 + \nu & 0
    \\
    0 & 0 & 0 & 0 & 0 & 1 + \nu
    \end{array}
\right)
\left(
    \begin{array}{c}
    \Delta \sigma_{xx}'
    \\
    \Delta \sigma_{yy}'
    \\
    \Delta \sigma_{zz}'
    \\
    \Delta \sigma_{xy}'
    \\
    \Delta \sigma_{yz}'
    \\
    \Delta \sigma_{xz}'
    \end{array}
\right)
\end{align}

bzw. nach den Spannungen aufgelöst

\begin{align}
\left(
    \begin{array}{c}
    \Delta \sigma_{xx}'
    \\
    \Delta \sigma_{yy}'
    \\
    \Delta \sigma_{zz}'
    \\
    \Delta \sigma_{xy}'
    \\
    \Delta \sigma_{yz}'
    \\
    \Delta \sigma_{xz}'
    \end{array}
\right)
=
\frac{E}{(1+\nu) (1-2\nu)}
\left(
    \begin{array}{cccccc}
    1 -\nu & \nu & \nu & 0 & 0 & 0
    \\
    \nu & 1 - \nu & \nu & 0 & 0 & 0
    \\
    \nu & \nu & 1 - \nu & 0 & 0 & 0
    \\
    0 & 0 & 0 & 1 - 2\nu & 0 & 0
    \\
    0 & 0 & 0 & 0 & 1 - 2\nu & 0
    \\
    0 & 0 & 0 & 0 & 0 & 1 - 2\nu
    \end{array}
\right)
\left(
    \begin{array}{c}
    \epsilon_{xx}
    \\
    \epsilon_{yy}
    \\
    \epsilon_{zz}
    \\
    \epsilon_{xy}
    \\
    \epsilon_{yz}
    \\
    \epsilon_{xz}
    \end{array}
\right)
\end{align}

Für die Verzerrungen wurde dabei auf die Ausweisung der Inkremente $\Delta$ verzichtet, da implizit angenommen wird, dass der Primärspannungszustand als unvervormter Ausgangszustand betrachtet wird.

## Verzerrungen im Halbraum

Uns interessieren nun vornehlich die Vertikalverschiebungen des Bodens an der Geländeoberkannte. Diese ergeben sich aus den sich über die Tiefe akkumulierenden Vertikaldehnungen:

$$
    s(x,y) = u_z(x,y)|_{z=0} = \int \limits_{z=0}^{z=\infty} \epsilon_{zz} \text{d}z
$$

Für die Vertikaldehnung finden wir die Beziehung

$$
    \epsilon_{zz} = \frac{1}{E} \left[ \Delta \sigma'_{zz} - \nu (\Delta \sigma'_{xx} + \Delta \sigma'_{yy})  \right],
$$

mit der Vertikalverformungen bei gegebenen Normalspannungsverteilungen berechnet werden können.

Im Sonderfall behinderter Seitendehnung, der oft unter ausgedehnten Lasten angenommen wird, ergeben sich die Horizontalspannungsinkremente zu

$$
    \Delta \sigma'_{xx} = \Delta \sigma'_{yy} = \frac{E \nu }{(1+\nu) (1-2\nu)} \epsilon_{zz}
$$

Eingesetzt in die obige Beziehung für die Vertikaldehnung ergibt sich dann

$$
    \epsilon_{zz} = \frac{\Delta \sigma'_{zz}}{E_\text{s}} \quad \text{mit} \quad  E_\text{s} = \frac{E(1-\nu)}{(1+\nu) (1-2\nu)}
$$

mit dem Steifemodul $E_\text{s}$ (Steifigkeit bei behinderter Seitendehnung). Im Folgenden wird implizit $\sigma'_{ij} = \sigma_{ij}$ angenommen.

## Isotroper elastischer Halbraum unter einer vertikalen Punktlast

Wir rufen uns die analytische Lösung nach Boussinesq in Erinnerung.

Die kartesischen Koordinaten $x,y,z$ ($z$ vertikal) haben ihren Ursprung am Angriffspunkt der Punktlast. Zusätzlich werden die horizontal radialen Abstände $r$ sowie der räumliche Abstant $R$ wie folgt definiert:

\begin{align}
    r &= \sqrt{x^2 + y^2}
    \\
    R &= \sqrt{r^2 + z^2}
\end{align}

Nach der analytischen Lösung von Boussinesq ergeben sich die folgenden Gleichungen für die drei Normalspannungsverläufe:

\begin{align}
    \Delta \sigma_{xx} &= \frac{3F_z}{2\pi R^2} \left[ \frac{x^2 z}{R^3} - \frac{1-2\nu}{3} \left( \frac{(x^2 - y^2) R}{r^2 (R + z)} + \frac{y^2 z}{R r^2}\right) \right]
    \\
    \Delta \sigma_{yy} &= \frac{3F_z}{2\pi R^2} \left[ \frac{y^2 z}{R^3} - \frac{1-2\nu}{3} \left( \frac{(y^2 - x^2) R}{r^2 (R + z)} + \frac{x^2 z}{R r^2}\right) \right]
    \\
    \Delta \sigma_{zz} &= \frac{3F_z}{2\pi R^2} \frac{z^3}{R^3}
\end{align}

Die Summe der beiden horizontalen Normalspannungen ergibt sich zu:

$$
    \Delta \sigma_{xx} + \Delta \sigma_{yy} = \frac{3F_z}{2\pi R^2} \left[ \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right]
$$

\begin{align}
    \epsilon_{zz} &= \frac{1}{E} \left[ \Delta \sigma'_{zz} - \nu (\Delta \sigma'_{xx} + \Delta \sigma'_{yy})  \right]
    \\
    &= \frac{3F_z}{2\pi R^2 E} \left[ \frac{z^3}{R^3} - \nu \left( \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right) \right]
\end{align}

Wir berechnen nun das Integral über $\epsilon_{zz}$ von $z=\infty$ bis zu einer Tiefe $\bar{z}$, also:

$$
    u_z(r,\bar{z}) = \frac{3F_z}{2\pi E} \int \limits_{\bar{z}}^{\infty} \frac{1}{R^2} \left[ \frac{z^3}{R^3} - \nu \left( \frac{r^2 z}{R^3} - \frac{1-2\nu}{3} \frac{z}{R} \right) \right] \text{d}z
$$

Dazu definieren wir in `sympy` den Integranden eps (ohne konstanten Vorfaktor) und den Radius R:

In [2]:
r, z, nu, eps = sp.symbols('r z nu epsilon')
R = sp.sqrt(r**2 + z**2)
R

   _________
  ╱  2    2 
╲╱  r  + z  

In [3]:
eps = 1/R**2 * ((z/R)**3 - nu * (r**2*z/R**3 - (1-2*nu)/3 * z/R))
eps

    ⎛                 ⎛1   2⋅ν⎞ ⎞               
    ⎜     2         z⋅⎜─ - ───⎟ ⎟         3     
    ⎜    r ⋅z         ⎝3    3 ⎠ ⎟        z      
- ν⋅⎜──────────── - ────────────⎟ + ────────────
    ⎜         3/2      _________⎟            3/2
    ⎜⎛ 2    2⎞        ╱  2    2 ⎟   ⎛ 2    2⎞   
    ⎝⎝r  + z ⎠      ╲╱  r  + z  ⎠   ⎝r  + z ⎠   
────────────────────────────────────────────────
                     2    2                     
                    r  + z                      

In [4]:
sp.integrate(eps,(z,z,sp.oo)).simplify()

     2  2      2  2      2      2      2
- 2⋅ν ⋅r  - 2⋅ν ⋅z  + ν⋅z  + 2⋅r  + 3⋅z 
────────────────────────────────────────
                        3/2             
               ⎛ 2    2⎞                
             3⋅⎝r  + z ⎠                

Geschickte Umformung durch Ausklammern und Verwendung der dritten binomischen Formel liefert:

\begin{align}
    u_z(r,z) &= \frac{3F_z}{2\pi E} \frac{1}{3R} \left[ 2(1-\nu^2) + \frac{z^2}{R^2} ( 1+\nu) \right]
    \\
    &= \frac{F_z}{2\pi E} \frac{1+\nu}{R} \left[ 2(1-\nu) + \frac{z^2}{R^2} \right]
\end{align}

Der Schubmodul ist mit dem E-Modul und der Querkontraktionszahl über

$$
    G = \frac{E}{2(1+\nu)}
$$

verknüpft, so dass unser Ergebnis letztlich lautet

$$
    u_z(r,z) = \frac{F_z}{4\pi G R} \left[ 2(1-\nu) + \frac{z^2}{R^2} \right]
$$

Oberflächensetzungen ergeben sich direkt zu

$$
    s(r) = u_z(r,z=0) = \frac{F_z(1-\nu)}{2\pi G r}
$$

Man erkennt in dieser Lösung wiederum die Singularität am Punkt des Lasteintrages.

Zum Vergleich berechnen wir nun noch die Vertikalverschiebung bei Verwendung der Annahme einachsiger Verformungen (bei der Punktlast eine fragwürdige Annahme).

$$
    \epsilon_{zz} = \frac{\Delta \sigma'_{zz}}{E_\text{s}} = \frac{3F_z}{2\pi E_\text{s} R^2} \frac{z^3}{R^3}
$$

Damit ergibt sich

$$
    \bar{u}_z(r,\bar{z}) = \frac{3F_z}{2\pi E_\text{s}} \int \limits_{\bar{z}}^{\infty} \frac{1}{R^2} \left[ \frac{z^3}{R^3} \right] \text{d}z
$$


In [5]:
eps = 1/R**2 * ((z/R)**3)
eps

      3     
     z      
────────────
         5/2
⎛ 2    2⎞   
⎝r  + z ⎠   

In [6]:
sp.integrate(eps,(z,z,sp.oo)).simplify()

    2       
 2⋅r     2  
 ──── + z   
  3         
────────────
         3/2
⎛ 2    2⎞   
⎝r  + z ⎠   

Umformung ergibt für die Vertikalverschiebung

$$
    \bar{u}(r,z) = \frac{F_z}{2\pi E_\text{s} R} \left[ 2 + \frac{z^2}{R^2} \right]
$$

und für die Setzung

$$
    \bar{s}(r) = \bar{u}(r,z=0) = \frac{F_z}{\pi E_\text{s} r}
$$

*Aufgabe*: Wie unterscheiden sich die Lösungen $s(r)$ und $\bar{s}(r)$ bei $\nu = 0$? Wie erklärt sich das?

Wir stellen nun beide Lösungen graphisch dar.

In [7]:
#HIDDEN
interactive_point_settlement()

interactive(children=(BoundedFloatText(value=100.0, description='$E$ / MPa', max=1000.0, min=0.001), BoundedFl…

Es treten größere Abweichungen auf, da unter der Punktlast nicht nur Kompressionssetzungen, sondern auch Verdrängungssetzungen auftreten.

## Isotroper elastischer Halbraum unter konstanter Rechtecklast

Aufgrund des Superpositionsprinzips kann die obige Lösung u.a. so integriert werden, dass die Eckpunktsetzungen einer $a \times b$ großen Rechtecklast der Intensität $q_0$ ermittelt werden können. Führt man die Integration des Vertikaldehnungsfeldes in den veränderten Grenzen 

$$
    s = \int \limits_{z=0}^{z=t} \epsilon_{zz} \text{d}z
$$

aus, also bis in eine endliche Tiefe $t$ durch, so entspricht das einer in der Tiefe $t$ anstehenden undeformierbaren Baugrundschicht. Die Eckpunktsetzung ergibt sich zu

$$
    s = \frac{q_0 (1-\nu^2)}{\pi E} \left[
        a \ln \frac{\left( b + \sqrt{a^2 + b^2} \right) \sqrt{a^2 + t^2}}{a(b+R)} + 
        b \ln \frac{\left( a + \sqrt{a^2 + b^2} \right) \sqrt{b^2 + t^2}}{b(a+R)}
    \right] + 
    \frac{q_0 (1-\nu-2\nu^2)}{2 \pi E} t \arctan \left( \frac{ab}{tR} \right)
$$

mit $R = \sqrt{a^2 + b^2 + t^2}$.

Den Einfluss der Integrationsgrenztiefe $t$ demonstriert die folgende Abbildung.

In [8]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_rect_settlement()

interactive(children=(BoundedFloatText(value=100.0, description='$E$ / MPa', max=1000.0, min=0.001), BoundedFl…

Oberflächennah anstehende sehr steife Baugrundschichten verringern die auftretenden Setzungen also sehr stark. Aufgrund der Spannungsausbreitung im Untergrund tragen tiefere Schichten immer weniger zum Setzungsbetrag bei. Zudem nimmt die Steifigkeit des Bodens mit der Tiefe zu. Die Schersteifigkeit bei sehr kleinen Deformationen ist zudem ebenfalls höher, als aus Versuchen mit größeren Deformationen zu erwarten wäre. Es lässt sich also die Existenz einer Grenztiefe für die Setzungsberechnung erwarten. Dazu das nächste Mal mehr.

## Mittelpunktsetzungen unter konstanten Rechteck bzw. Kreislasten

Für Mittelpunktsetzungen unter schlaffen Lasten $q_0$ auf kreisförmigen Flächen des Radius $R$ gilt nach *Fischer* 

$$
    s_{\text{M; Kreis}}(R) = \frac{2q_0}{E} \left[ (1-\nu^2) \left( R + t - \sqrt{t^2 + R^2} \right) - \frac{t(1+\nu)}{2} \left( 1 - \frac{t}{\sqrt{t^2 + R^2}} \right) \right]
$$

Die Mittelpunktsetzung der Rechtecklast berechnen wir aus der bereits bekannten Eckpunktformel durch Superposition:

$$
    s_{\text{M; Rechteck}}(a \times b) = 4 s_{\text{E; Rechteck}}\left( \frac{a}{2} \times \frac{b}{2} \right)
$$

In der folgenden Abbildung werden die Setzungen unter Rechtecklasten (Eck- und Mittelpunkte) mit denen unter flächengleichen Kreislasten verglichen.

In [9]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_rect_circ_settlement()

interactive(children=(BoundedFloatText(value=100.0, description='$E$ / MPa', max=1000.0, min=0.001), BoundedFl…

## Zur Grenztiefe

Wir nähern uns nun der Fragestellung einer Grenztiefe für die Setzungsermittlung und nutzen dafür die bekannten Formeln für die Eckpunkte von Rechtecklasten. Wir betrachten einerseits das Verhältnis des berechneten Setzungswertes bei Ansatz einer bestimmten Grenztiefe $t$ und des Grenzwertes bei unendlicher Grenztiefe:

$$
    \frac{s(t)}{\lim \limits_{t \rightarrow \infty} s(t)}
$$

Dieses Verhältnis vergleichen wir mit dem Verhältnis der auflastinduzierten Spannungsänderung $\Delta \sigma_{zz}$ zum Betrag der Vertikalspannung im Grundspannungszustand $\sigma_{0,zz}$ in:

$$
    \frac{\Delta \sigma_{zz}(t)}{\sigma_{0,zz}(t)} \quad \text{mit} \quad \sigma_{0,zz}(z) = \gamma t
$$

In [10]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_grenztiefe()

interactive(children=(BoundedFloatText(value=3.0, description='$a$ / m', max=10.0, min=0.1), BoundedFloatText(…

Der reale Baugrund ist kein unendlicher Halbraum mit konstanten Eigenschaften. Die Steifigkeit des Bodens nimmt i.d.R. mit zunehmender Tiefe stark zu (es können aber auch stark kompressible und damit setzungsgebende Schichten anstehen). Zudem sind die Steifigkeiten bei sehr kleinen Verformungen deutlich höher als die Werte, die aus den typischen Kompressionsversuchen (Ödometer, Triax) bei den dort üblichen Deformationen ermittelt werden. Die mit der obigen Formel ermittelten Setzungsbeträge sind daher in der Regel unrealistisch. Die Schichtung des Baugrundes mit ihren variablen Eigenschaften ist daher in den Berechnungen zu berücksichtigen.

Wenn keine stark zusammendrückbaren Schichten ein anderes Vorgehen nahelegen, wird die Grenztiefe der Setzungsberechnung als die Dicke des die tatsächlich auftretenden Setzungen bestimmenden Bereichs durch das Kriterium $\Delta \sigma_{zz}(t_\text{s}) \leq 0.2 \sigma_{0,zz}$ festgelegt.

Details zur Festlegung der Grenztiefe siehe Vorlesungsunterlagen und DIN 4019.

## Formulierung in Beiwerten

Man kann die bisherige Beziehung

$$
    s = \frac{q_0 (1-\nu)^2}{\pi E} \left[
        a \ln \frac{\left( b + \sqrt{a^2 + b^2} \right) \sqrt{a^2 + t^2}}{a(b+R)} + 
        b \ln \frac{\left( a + \sqrt{a^2 + b^2} \right) \sqrt{b^2 + t^2}}{b(a+R)}
    \right] + 
    \frac{q_0 (1-\nu-2\nu^2)}{2 \pi E} t \arctan \left( \frac{ab}{tR} \right)
$$

auch mit Hilfe des dimensionslosen Beiwertes $f_\text{R}$ formulieren:

\begin{align*}
    s &= \frac{q_0 b f_\text{R}}{E}
    &\text{mit } f_\text{R} = \frac{(1-\nu)^2}{b \pi} \left[
        a \ln \frac{\left( b + \sqrt{a^2 + b^2} \right) \sqrt{a^2 + z^2}}{a(b+R)} + 
        b \ln \frac{\left( a + \sqrt{a^2 + b^2} \right) \sqrt{b^2 + z^2}}{b(a+R)}
    \right] + 
    \frac{(1-\nu-2\nu^2)}{2 b \pi} t \arctan \left( \frac{ab}{zR} \right)
\end{align*}

wobei die Dicke der zusammendrückbaren Schicht hier mit $z$ bezeichnet wurde.

Für die baupraktische Berechnung wird nun ein Rechenmodul $E^*$ eingeführt, der die Zusammendrückbarkeit des Materials charakterisiert. Für homogene Verhältnisse kann $E^* = E/(1-\nu^2)$ abgeleitet werden. Allerdings hat diese Beziehung praktisch kaum Bedeutung und $E^*$ wird aus anderen Überlegungen ermittelt. Setzt man in die obigen Beziehungen $\nu \leftarrow 0$ und $E \leftarrow E^*$, so gelangt man zu folgender Formulierung mit der beiwertbasierten Beziehung, die innerhalb der Lastfläche mit ausreichender Genauigkeit Verwendung findet:

\begin{align*}
    s &= \frac{q_0 b f_\text{R}}{E^*}
    \\
    &\text{mit } f_\text{R} = \frac{1}{b \pi} \left[
        a \ln \frac{\left( b + \sqrt{a^2 + b^2} \right) \sqrt{a^2 + z^2}}{a(b+R)} + 
        b \ln \frac{\left( a + \sqrt{a^2 + b^2} \right) \sqrt{b^2 + z^2}}{b(a+R)}
    \right] + 
    \frac{1}{2 b \pi} t \arctan \left( \frac{ab}{zR} \right)
    \\
    &\text{und }  R = \sqrt{a^2 + b^2 + z^2}
\end{align*}

Zur einfacheren Handhabung werden Bemessungsdiagramme eingeführt:

In [11]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_fR_rechteck()

interactive(children=(BoundedFloatText(value=1.0, description='$z$ / $b$', min=0.1), BoundedFloatText(value=2.…

Zum Vergleich werden die Kurven für $\nu \neq 0$ geplottet, die bei Punkten außerhalb der Lastfläche Anwendung finden. Es sei darauf hingewiesen, dass die Verwendung mit $s = q_0 b f_\text{R}/E^*$ nur für $E^* \leftarrow E$ der mechanisch korrekten Lösung entspricht. Man verfolgt mit der Einführung der generalisierten Rechenmoduln allerdings das Ziel, das Einsetzen von erfahrungsbasiert korrigierten Werten in der Normung zu ermöglichen. In der folgenden Abbildung sind die gestrichelten Linien für $\nu = 0$ zum Vergleich dargestellt.

In [12]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_fR_rechteck_nu()

interactive(children=(BoundedFloatText(value=1.0, description='$z$ / $b$', min=0.1), BoundedFloatText(value=2.…

Durch Anwendung der Formel für $\nu=0$ können Setzungen an beliebigen Punkten im Inneren der Lastfläche ermittelt werden. Von besonderem Interesse ist der sog. kennzeichnende Punkt, an dem die Setzungen einer Setzungsmulde unter einer schlaffen Last gleich denen unter einer starren Last mit gleicher Resultierender sind. Definiert man auch hier 

$$
    s_\text{K} = \frac{q_0bf_\text{K}}{E^*}
$$

so gilt demzufolge

$$
    f_\text{K} = \sum \limits_{i=0}^3 f_\text{R}(a_i, b_i) \frac{b_i}{b}
$$

mit den Koordinaten des kennzeichnenden Punktes (Eckpunkte der vier superponierten Rechtecke)

\begin{align}
    x_i &= \{0.87a,\ 0.87a,\ 0.87b,\ 0.13a\} \quad \text{für } i \in [0,3]
    \\
    y_i &= \{0.87b,\ 0.13b,\ 0.13a,\ 0.13b\} \quad \text{für } i \in [0,3]
\end{align}

In [13]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interactive_fK_rechteck_nu()

interactive(children=(BoundedFloatText(value=1.0, description='$z$ / $b$', min=0.1), BoundedFloatText(value=2.…

## Anwendung auf geschichteten Baugrund

Für die Erweiterung auf geschichteten Baugrund werden die Setzungsbeträge schichtweise berechnet und aufaddiert. Die nachfolgende Skizze veranschaulicht das Prinzip:

<img src="https://github.com/nagelt/Teaching_Scripts/raw/master/Images/settlement_layers.png" style="width: 1200px;"/>

Zur Anwendung kommt folgende Formel:

$$
    s = q_0 b \left( \frac{f_1}{E_1^*} + \sum \limits_{i=2}^n \frac{f_i - f_{i-1}}{E_i^*} \right)
$$

Der laufende Index $i$ kennzeichnet die Schichtnummer ($i=1$: oberste Schicht), $f_i$ bezeichnet den Setzungsbeiwert der $i$-ten Schicht bezogen auf ihre Unterkante und $E_i^*$ ist der mittlere Zusammendrückungsmodul (Rechenmodul) der $i$-ten Schicht.

Die folgenden Skripte können zur Übung von Beispielen genutzt werden:

1. Anzahl der Bodenschichten bestimmen und anschließend 'update_layers' anklicken (funktioniert nur in Jupyter. In HTML Version auf 5 Bodenschichten festgelegt; 'update_layers' entfällt).
2. Die Sohlnormalspannung $q_0$, die Tiefe $d$ und mittlere Wichte $\gamma_d$ des Aushubs, die Querkontraktionszahl $\nu$, sowie die Fundamentabmessungen eingeben.
3. Die Tiefen der Schichtunterkanten gemessen ab Baugrundsohle ($z_i$), die Rechenmodule der Schichten $E_i^*$ und deren Wichten $\gamma_i$ (unter Auftrieb oder total) angeben.
3. 'update_analysis' anklicken und Ergebnis anhand der graphischen Darstellung auswerten.

In [14]:
#HIDDEN
#global variables
#%run plot_functions/settlements_plots.ipynb
html = True
items = []
if html:
    n_layers = widgets.BoundedIntText(value=5,min=5,max=5,description='$n_\\mathrm{layers}$')
    print('In the HTML version you can only work with 5 layers.')
else:
    n_layers = widgets.BoundedIntText(value=5,min=1,max=10,description='$n_\\mathrm{layers}$')
n_layers

In the HTML version you can only work with 5 layers.


BoundedIntText(value=5, description='$n_\\mathrm{layers}$', max=5, min=5)

In [15]:
#HIDDEN
if html:
    items = [widgets.BoundedFloatText(description='$q_0$ / kPa',value=300,min=0,max=2000),
             widgets.BoundedFloatText(description='$d$ / m',value=2,min=0,max=100),
             widgets.BoundedFloatText(description='$\\gamma_d$ / kN m$^{-3}$',value=18,min=0,max=30),
             widgets.BoundedFloatText(description='$\\nu$',value=0,min=0,max=0.499),
             widgets.BoundedFloatText(description='$a$ / m',value=2,min=0.01,max=50),
             widgets.BoundedFloatText(description='$b$ / m',value=2,min=0.01,max=50)]
    for i in range(n_layers.value):
        items.append(widgets.BoundedFloatText(description='$z_{%i}$ / m' %(i+1),value=1+i*2,min=0,max=200))
        items.append(widgets.BoundedFloatText(description='$E^*_{%i}$ / MPa' %(i+1),value=(i+1)*10,min=0.01,max=10000))
        items.append(widgets.BoundedFloatText(description='$\gamma_{%i}$ / kN m$^{-3}$' %(i+1),value=18-i,min=1,max=35))
    w = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3,350px)"))
else:
    w = interact(run_layers, update_layers=True);

display(w);

GridBox(children=(BoundedFloatText(value=300.0, description='$q_0$ / kPa', max=2000.0), BoundedFloatText(value…

In [16]:
#HIDDEN
#%run plot_functions/settlements_plots.ipynb
interact(run_analysis, update_analysis=True);

interactive(children=(Checkbox(value=True, description='update_analysis'), Output()), _dom_classes=('widget-in…

## Spannungsänderung durch GW Absenkung oder Anstieg

Die folgende Abbildung verdeutlicht die Veränderung neutraler, effektiver und totaler Spannungen infolge variabler Grundwasserstände für ausgewählte Wichten und einen mit der Tiefe zunehmenden Steifemodul nach Ohde. Die Zahlenwerte sind als exemplarisch zu erachten. Man beachte die Relevanz der tiefenabhängigen Steifigkeit und des Konzeptes der Grenztiefe, um bei großräumigen Spannungsänderungen wie in diesem Fall zu sinnvollen Ergebnissen zu gelangen. Man beachte auch, dass bei Grundwasserspiegeländerungen Sackungen und ähnliche Phänomene auftreten können, die zu Deformationen führen, die mit dem unten verwendeten einfachen Modell nicht erklärbar sind und qualitativ falsch abgeschätzt werden könnten.

In [17]:
interact(stress_depth, dzw = widgets.FloatSlider(min=-2, max=4, value=2,description='$\\Delta z_\\mathrm{GW}$ / m'));

interactive(children=(FloatSlider(value=2.0, description='$\\Delta z_\\mathrm{GW}$ / m', max=4.0, min=-2.0), O…